In [1]:
import torch
#assert '.'.join(torch.__version__.split('.')[:2]) == '1.4'
import torch.nn as nn
import torch.nn.functional as F  # useful stateless functions
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\Users\Admin\AppData\Local\Programs\Python\Python310\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cpu


In [3]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image
class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

In [4]:
NUM_TRAIN = 49000


transform = T.Compose([
                T.ToTensor(),
                T.Resize((16,16)),
                T.Grayscale()
            ])


rice_data = dset.ImageFolder('./CV7062610/datasets/rice/Rice_Image_Dataset',
                             transform=transform)

rice_train, rice_val,rice_test = random_split(rice_data,[0.6,0.25,0.15],torch.Generator().manual_seed(13))

loader_train = DataLoader(rice_train, batch_size=64)

loader_val = DataLoader(rice_val, batch_size=64)


loader_test = DataLoader(rice_test, batch_size=64)



In [5]:
import itertools

#returns an array with the correct nn.Modules for a single model using our architecture
def makeModel(in_shape,out_classes,N,K,M,numFilters,filterSizes,poolSizes,hiddenDims,convNorm=False,affineNorm=False,convDrop=0,affineDrop=0):
    model = []
    #convolution and pooling
    for n in range(N):
        for k in range(K):
            
            model.append(nn.Conv2d(in_shape[0],numFilters[n][k],(filterSizes[n][k],filterSizes[n][k]),padding = filterSizes[n][k]//2))
            in_shape[0] = numFilters[n][k]
            if convNorm:
                model.append(nn.BatchNorm2d(in_shape[0]))
            model.append(nn.ReLU())
            if convDrop > 0:
                model.append(nn.Dropout2d(convDrop))
        model.append(nn.MaxPool2d(poolSizes[n]))
        in_shape[1] = 1 + ((in_shape[1]-poolSizes[n])//poolSizes[n])
        in_shape[2] = 1 + ((in_shape[2]-poolSizes[n])//poolSizes[n])
        

    #prep for affine layers
    model.append(Flatten())
    in_shape = in_shape[0]*in_shape[1]*in_shape[2]

    #affine layers
    for m in range(M-1):
        
        model.append(nn.Linear(in_shape,hiddenDims[m]))
        in_shape = hiddenDims[m]
        if affineNorm:
            model.append(nn.BatchNorm1d(in_shape))
        model.append(nn.ReLU())
        if affineDrop>0:
            model.append(nn.Dropout(affineDrop))


    #we always add an extra affine layer
    model.append(nn.Linear(in_shape,out_classes))
    if affineNorm:
        model.append(nn.BatchNorm1d(out_classes))
    model.append(nn.ReLU())
    if affineDrop>0:
        model.append(nn.Dropout(affineDrop))

    return model


#returns all the permutations of our paramaters
def generate_model_params(param_grid):
    
    params1 = []
    #generate all the possible combinations of number of filters, filter sizes, hidden dims and pool sizes
    for N in param_grid["N"]:
        for K in param_grid["K"]:
            for M in param_grid["M"]:
                for numFilters in itertools.product(param_grid["numFilters"], repeat=N*K):
                    for filterSizes in itertools.product(param_grid["filterSizes"], repeat=N*K):
                        for hiddenDims in itertools.product(param_grid["hiddenDims"], repeat=M):
                            for poolSizes in itertools.product(param_grid["poolSizes"], repeat=N):
                                
                                params = {
                                    'N': N,
                                    'K': K,
                                    'M': M,
                                    'numFilters': [[numFilters[i*K + j] for j in range(K)] for i in range(N)],
                                    'filterSizes': [[filterSizes[i*K + j] for j in range(K)] for i in range(N)],
                                    'poolSizes': list(poolSizes),
                                    'hiddenDims': list(hiddenDims),
                                    
                                }
                                
                                params1.append(params)

    #rest of the params
    params = {
        'convNorm': param_grid['convNorm'],
        'affineNorm': param_grid['affineNorm'],
        'convDrop': param_grid['convDrop'],
        'affineDrop': param_grid['affineDrop']
    }
    keys, values = zip(*params.items())
    params2 = [dict(zip(keys, v)) for v in itertools.product(*values)]


    #combine the 2
    param_combinations = []
    for param1 in params1:
        for param2 in params2:
            
            param_combinations.append({**param1,**param2})

    return param_combinations

In [6]:
#similar to part34 but returns accuracy
def check_accuracy_part5(loader, model,verbose=0):
    
    
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        if verbose>1:
            print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
        return acc
    

#similar to part34 but returns train and validation accuracy
def train_part5(model, optimizer,lossFunction=F.cross_entropy, epochs=1, verbose = 0, print_every=100):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: train accuracy, validation accuracy
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = lossFunction(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                if verbose>1:
                    print('Iteration %d, loss = %.4f' % (t, loss.item()))
                    check_accuracy_part5(loader_val, model,verbose=verbose)

    train_acc = check_accuracy_part5(loader_train,model,verbose=verbose)
    val_acc = check_accuracy_part5(loader_val, model,verbose=verbose)
    if verbose > 0:    
        print('train loss = %.4f' % (loss.item()))
    if verbose == 1:
        print('train accuracy: (%.2f), val accuracy: (%.2f)' % (100 * train_acc,100*val_acc))

    return train_acc,val_acc
                

In [7]:
#note, adding paramaters here can greatly increase run time
param_grid = {
    
    'N': [1,2],
    'K': [2,3],
    'M': [2],
    'numFilters': [8],
    'filterSizes': [5],
    'poolSizes': [2],
    'hiddenDims': [64],
    'convNorm': [True],
    'affineNorm': [True],
    'convDrop': [0],
    'affineDrop': [0]
}
params = generate_model_params(param_grid)
len(params)

4

In [8]:
#generate all the models
models = {}
for i in range(len(params)):
    
    models[i]= [makeModel([1,16,16],5,**params[i]),params[i]]

In [9]:
################################################################################
# TODO:                                                                        #         
# Experiment with any architectures, optimizers, and hyperparameters.          #
# Achieve AT LEAST 70% accuracy on the *validation set* within 10 epochs.      #
#                                                                              #
# Note that you can use the check_accuracy function to evaluate on either      #
# the test set or the validation set, by passing either loader_test or         #
# loader_val as the second argument to check_accuracy. You should not touch    #
# the test set until you have finished your architecture and  hyperparameter   #
# tuning, and only run the test set once at the end to report a final value.   #
################################################################################
model = None
optimizer = None

# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

#we can check different learning rates and momentums 
optimizers = {1:[optim.SGD,{"lr":3e-2,"momentum":0.9, "nesterov":True}]}

res = {}
best_val = -1
best_model = None

#iterate over all optimizers and models
for k1,net in models.items():
    model = nn.Sequential(*net[0])
    res[k1] = {}
    for k2,opt in optimizers.items():
        optimizer = opt[0](model.parameters(),**opt[1])
        
        print(net[1])
        train_acc, val_acc = train_part5(model, optimizer,epochs=2,verbose=1)
        print("--------------------------------------------------------------------")
        res[k1][k2] = [val_acc,train_acc,net[1]]
        #keep the best one
        if val_acc > best_val:
            best_val = val_acc
            best_model = model
            best_params = net[0]
            best_optimizer = optimizer


#finally set model and optimizer to be the best ones
model = nn.Sequential(*best_params)
optimizer = best_optimizer



# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             
################################################################################
train_part5(model, optimizer, epochs=2)
best_model = model
check_accuracy_part5(loader_test, best_model)


{'N': 1, 'K': 2, 'M': 2, 'numFilters': [[8, 8]], 'filterSizes': [[5, 5]], 'poolSizes': [2], 'hiddenDims': [64, 64], 'convNorm': True, 'affineNorm': True, 'convDrop': 0, 'affineDrop': 0}


c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


train loss = 0.3909
train accuracy: (95.48), val accuracy: (95.05)
--------------------------------------------------------------------
{'N': 1, 'K': 3, 'M': 2, 'numFilters': [[8, 8, 8]], 'filterSizes': [[5, 5, 5]], 'poolSizes': [2], 'hiddenDims': [64, 64], 'convNorm': True, 'affineNorm': True, 'convDrop': 0, 'affineDrop': 0}
train loss = 0.3152
train accuracy: (94.92), val accuracy: (94.49)
--------------------------------------------------------------------
{'N': 2, 'K': 2, 'M': 2, 'numFilters': [[8, 8], [8, 8]], 'filterSizes': [[5, 5], [5, 5]], 'poolSizes': [2, 2], 'hiddenDims': [64, 64], 'convNorm': True, 'affineNorm': True, 'convDrop': 0, 'affineDrop': 0}
train loss = 0.3724
train accuracy: (94.05), val accuracy: (93.93)
--------------------------------------------------------------------
{'N': 2, 'K': 3, 'M': 2, 'numFilters': [[8, 8, 8], [8, 8, 8]], 'filterSizes': [[5, 5, 5], [5, 5, 5]], 'poolSizes': [2, 2], 'hiddenDims': [64, 64], 'convNorm': True, 'affineNorm': True, 'convDrop'

0.9575111111111111

In [10]:
torch.save(best_model, "model2.pt")